<a href="https://colab.research.google.com/github/RobinDoughnut/karpathy-neural-networks/blob/main/PyTorch_buildNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data. The torch.nn namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.



In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Get Device for Training


We want to be able to train our model on an accelerator such as CUDA, MPS, MTIA, or XPU.
If the current accelerator is available, we will use it. Otherwise, we use the CPU.



In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


### Define the Class

We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__.
Every nn.Module subclass implements the operations on input data in the forward method.



In [3]:
import torch.nn as nn
import torch.nn.functional as F

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call model.forward() directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the nn.Softmax module.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5], device='cuda:0')


### Model Layers

Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of
size 28x28 and see what happens to it as we pass it through the network.



In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

We initialize the nn.Flatten layer to convert each 2D 28x28 image into
a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained).



In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear


The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU
Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use nn.ReLU between our linear layers, but there’s other activations to introduce non-linearity in your model.



In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.7547,  0.4732, -0.2313,  0.1594, -0.3064, -0.3027, -0.5200, -0.2161,
          0.3825,  0.0066, -0.0117,  0.5782, -0.1564, -0.7939,  0.1596,  0.5934,
         -0.6945,  0.5072,  0.2164, -0.1349],
        [-0.8367, -0.0680, -0.1166,  0.2008, -0.2912, -0.0865, -0.2417,  0.0156,
          0.5541,  0.3268, -0.1589,  0.7749, -0.0321, -1.0789,  0.0974,  0.6628,
         -0.4135,  0.0370,  0.2536, -0.2504],
        [-0.6736, -0.0564, -0.4477, -0.1088, -0.0507, -0.0034, -0.2096,  0.2399,
          0.3670,  0.6365, -0.2558,  0.6545, -0.1524, -0.8549,  0.1987,  0.4429,
         -0.4848,  0.2454,  0.4417, -0.1282]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.4732, 0.0000, 0.1594, 0.0000, 0.0000, 0.0000, 0.0000, 0.3825,
         0.0066, 0.0000, 0.5782, 0.0000, 0.0000, 0.1596, 0.5934, 0.0000, 0.5072,
         0.2164, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.2008, 0.0000, 0.0000, 0.0000, 0.0156, 0.5541,
         0.3268, 0.0000, 0.7749, 0.0000, 0.0000, 0.09

### nn.Sequential
nn.Sequential is an ordered container of modules.
The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like seq_modules.



In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

### nn.Softmax
The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module.
The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class.
dim parameter indicates the dimension along which the values must sum to 1.


In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### Model Parameters
Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s parameters() or named_parameters() methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0335,  0.0297, -0.0333,  ...,  0.0064, -0.0325,  0.0232],
        [-0.0216, -0.0190, -0.0045,  ...,  0.0173,  0.0355,  0.0127]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0157, 0.0256], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0197,  0.0001,  0.0046,  ...,  0.0176,  0.0118, -0.0394],
        [ 0.0035, -0.0174,  0.0078,  ...,  0.0140, -0.0110, -0.0040]],
       device='cuda:0', grad_fn=<Slic